# SO-100 Teleoperation
Run cells top to bottom. Ports are auto-detected so it works regardless of USB enumeration order.

## 1. Detect Ports
Reads motor voltage to identify leader (~5V USB) vs follower (~12V PSU) on /dev/ttyACM0 and /dev/ttyACM1.

In [1]:
import scservo_sdk as scs

PORTS = ["/dev/ttyACM0", "/dev/ttyACM1"]
VOLTAGE_ADDR = 62  # Present_Voltage, 1 byte, unit = 0.1V

leader_port = None
follower_port = None

for port_path in PORTS:
    port = scs.PortHandler(port_path)
    if not port.openPort():
        print(f"{port_path}: CANNOT OPEN")
        continue
    port.setBaudRate(1000000)
    ph = scs.PacketHandler(0)

    data, comm, _ = ph.readTxRx(port, 1, VOLTAGE_ADDR, 1)
    port.closePort()

    if comm != 0:
        print(f"{port_path}: READ ERROR (comm={comm})")
        continue

    volts = data[0] / 10.0
    if volts > 8:
        label = "FOLLOWER (~12V)"
        follower_port = port_path
    else:
        label = "LEADER  (~5V)"
        leader_port = port_path

    print(f"{port_path}: {volts:.1f}V  -->  {label}")

if leader_port and follower_port:
    print("\nPorts detected OK!")
else:
    print("\nERROR: Could not find both arms! Are they plugged in?")

/dev/ttyACM0: 12.4V  -->  FOLLOWER (~12V)
/dev/ttyACM1: 5.2V  -->  LEADER  (~5V)

Ports detected OK!


## 2. Choose Mode and Run

In [ ]:
# Choose your mode (uncomment ONE):
#MODE = "basic"              # No camera
MODE = "wrist_only"       # Innomaker wrist cam only
#MODE = "top_only"         # ProXtend top/overhead only
#MODE = "top_wrist"        # Both cameras

# Camera indices (run `v4l2-ctl --list-devices` if these change):
#   0 = Innomaker 1080p  — wrist camera (mounted on arm)
#   4 = ProXtend USB     — top/overhead view

camera_configs = {
    "basic": "",
    "wrist_only": "--robot.cameras='{ wrist: {type: opencv, index_or_path: 0, width: 640, height: 480, fps: 30} }' --display_data=true",
    "top_only": "--robot.cameras='{ top: {type: opencv, index_or_path: 4, width: 640, height: 480, fps: 30} }' --display_data=true",
    "top_wrist": "--robot.cameras='{ top: {type: opencv, index_or_path: 4, width: 640, height: 480, fps: 30}, wrist: {type: opencv, index_or_path: 0, width: 640, height: 480, fps: 30} }' --display_data=true",
}

assert leader_port and follower_port, "Run cell 1 first! Ports not detected."

cmd = (
    f"lerobot-teleoperate"
    f" --robot.type=so100_follower --robot.port={follower_port} --robot.id=my_follower"
    f" --teleop.type=so100_leader --teleop.port={leader_port} --teleop.id=my_leader"
    f" {camera_configs[MODE]}"
)

print(f"Mode: {MODE}")
print(f"Command:\n\n{cmd}\n")

Mode: top_wrist
Command:

lerobot-teleoperate --robot.type=so100_follower --robot.port=/dev/ttyACM0 --robot.id=my_follower --teleop.type=so100_leader --teleop.port=/dev/ttyACM1 --teleop.id=my_leader --robot.cameras='{ top: {type: opencv, index_or_path: 4, width: 640, height: 480, fps: 30}, wrist: {type: opencv, index_or_path: 0, width: 640, height: 480, fps: 30} }' --display_data=true



In [ ]:
# Run teleoperation (Ctrl+C or interrupt kernel to stop)
import subprocess, shlex, sys

print("Starting teleoperation... (interrupt kernel to stop)\n")
process = subprocess.Popen(shlex.split(cmd), stdout=sys.stdout, stderr=sys.stderr)
process.wait()

Starting teleoperation... (interrupt kernel to stop)

